<img src='https://eurekadocs.readthedocs.io/en/latest/_images/Eureka_logo.png' alt="eureka_logo" width="400px"/><img src='https://exoclimes.org/img/exoslam-bg.png' alt="ariel_france_logo" width="220px"/>

# ExoSLAM 2025
## Bonus Tutorial - Eureka Fitting of MIRI/LRS Data

**Authors**: Taylor James Bell (ESA/AURA for STScI)<br>
**Last Updated**: June 25, 2025<br>
**jwst Pipeline Version**: 1.18.0 (Build 11.3)<br>
**Eureka! Pipeline Version**: [exoslam2025](https://github.com/kevin218/Eureka/tree/exoslam2025) (based on Eureka! version 1.2.1)

**Purpose**:<br/>

The objective of this notebook is to gain familiarity with how space-telescope spectroscopy is "analyzed", which is a term often used to describe the process of going from measurements of the system's brightness as a function of time (called spectroscopic lightcurves) to planetary transmission or emission spectra.

**Data**:<br/>
This notebook is set up to use an example dataset is from [Program ID](https://www.stsci.edu/jwst/science-execution/program-information) 1366 (PI: Batalha, Natalie) which is the JWST Transiting Exoplanet Community ERS program. In particular, we will use the MIRI/LRS full-orbit phase curve of the hot Jupiter WASP-43b, which was first published in [Bell et al. (2024)](https://ui.adsabs.harvard.edu/abs/2024NatAs...8..879B/abstract). These observations continuously monitored the WASP-43 system for 26.5 hours and contain two eclipses of WASP-43b, one transit of WASP-43b, and the orbital phase variations caused by changes in the regions of WASP-43b's atmosphere that were pointed toward JWST throughout the planet's orbit.

For our purposes, we are only going to work on the first 6 segments of the first exposure to keep things fairly speedy while still getting a realistic sense of what it is like to reduce MIRI/LRS data.

**JWST pipeline version and CRDS context**:<br/>
This notebook was written for the calibration pipeline version given above and uses the context associated with this version of the JWST Calibration Pipeline. Information about this and other contexts can be found in the JWST Calibration Reference Data System (CRDS) [server]((https://jwst-crds.stsci.edu/)). If you use different pipeline
versions, please refer to the table [here](https://jwst-crds.stsci.edu/display_build_contexts/) to determine what context to use. To learn more about the differences for the pipeline, read the relevant [documentation](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline/jwst-operations-pipeline-build-information)

**Eureka! pipeline version**:<br/>
Eureka is a pipeline developed to reduce and analyze HST and JWST data ([Bell et al. 2023](https://joss.theoj.org/papers/10.21105/joss.04503.pdf)). 
The code is available on [Github](https://github.com/kevin218), and there is also detailed [online documentation](eurekadocs.readthedocs.io).
This notebook was written for the particular Eureka! version specified above. If you are not using that particular version, you must ensure that all of your ECF and EPF settings are adjusted as necessary (looking at the demos/JWST folder for the Eureka! version you have installed is a good way to check whether any parameters changed)

## Table of Contents

* [0. Configuration](#0.-Configuration)
* [5. Sampling white lightcurve posterior](#5.-Sampling-white-lightcurve-posterior)

---
## 0. Configuration

The first step is to setup the notebook and environment.

We'll first import Eureka! along with some other useful packages.

In [ ]:
import batman
import eureka
import os
import numpy as np

Next, we will re-use the eventlabel from Notebook #1. The eventlabel is a short, meaningful label (without spaces) that describes the data you're currently working on. For simplicity, simply set `eventlabel = 'miri_exoslam'`. This same event label should be used throughout all stages.

In [ ]:
eventlabel = ''  # <--- Please update this as described above

Please adjust the following variable to specify specify where you downloaded the data in the Setup.ipynb notebook!

In [ ]:
path_to_data_folder_on_your_machine = '../data/'  # <--- Please update this variable if needed to match your data location
path_to_data_folder_on_your_machine = os.path.expanduser(path_to_data_folder_on_your_machine)

---
### An introduction to Bayesian statistics

*The text discussing Bayesian statistics throughout this notebook is adapted from:*
<blockquote>Bell, Taylor James. "Characterizing Ultra-Hot Jupiters through Theoretical Modelling and Precise Observations." Doctoral Thesis, McGill University, 2021.</blockquote>

<br/>

**This introduction to Bayesian statistics can be skipped if you are already familiar, and you can resume at "Optimizing a fit to the white lightcurve".**

<br/>

This workshop will make heavy use of Bayesian statistics which is a fundamental aspect of most aspects of modern science and data analysis. According to Bayes' theorem, the probability, $P$, of a hypothesis, $H$, given a set of observations, $\mathbf{X}$, and a collection of prior knowledge, $I$, is given by
\begin{equation*}
    P(H|\mathbf{X},I) = \frac{P(\mathbf{X}|H,I) P(H|I)}{P(\mathbf{X}|I)},
\end{equation*}
where $|$ reads as "given" (that all terms to the right are assumed true), and the comma reads as "and". The power of Bayes' theorem is that it allows us to compute the *posterior probability*, $P(H|\mathbf{X},I)$, using the much more easily calculable *likelihood function*, $P(\mathbf{X}|H,I)$, which is the probability that we would have observed the data $\mathbf{X}$ if the hypothesis and prior knowledge were correct. The $P(H|I)$ term is the *prior probability* and summarizes how our prior knowledge affects our hypothesis before having measured the data $\mathbf{X}$. Finally, the $P(\mathbf{X}|I)$ term is the *evidence* or *marginal likelihood* and is often omitted when fitting a model to data as it is only a normalization term and does not depend on the hypothesis.

When fitting a set of observations, a hypothesis typically consists of a function describing the model which depends on a collection of parameters, $\theta$, and hyperparameters, $\alpha$. Bayes' theorem can then be re-written as
\begin{equation*}
    P(\theta|\mathbf{X},\alpha) \propto P(\mathbf{X}|\theta,\alpha) P(\theta|\alpha).
\end{equation*}
Fitting the observations usually starts by freezing the set of hyperparameters and then evaluating the posterior probability by comparing different model predictions to the observed data. Fitting the observations then requires determining the values of $\theta$ that maximize the posterior probability (called the Maximum A Posteriori estimate or MAP estimate), while determining the uncertainty on the model parameters involves determining the range of values of $\theta$ that provide an adequately good fit to the observations (called the confidence interval).

In principle, one could simultaneously estimate the optimal value of $\theta$ and its confidence interval from the posterior probability density function (PDF) by calculating the posterior probability for all values of $\theta$; this is called a grid search. While this technique may be feasible for discrete parameters or low dimensional problems, performing a grid search when the vector $\theta$ contains tens or thousands of continuous variables becomes immensely challenging and computationally inefficient. Instead, various algorithms can be used to compute the MAP estimate and the confidence interval, some of which we will make use of in this workshop.

---
## 1. Sampling white lightcurve posterior

A sampling algorithm like the Markov Chain Monte-Carlo (MCMC) method is typically used to estimate parameter best-fit values and uncertainties. Monte-Carlo (MC) methods in general involve randomly sampling values of $\theta$, while MCMCs are a specific variant where samples are randomly drawn based on the knowledge of the posterior probability from the previously drawn value; the draws of MCMC are typically called steps taken by a walker, and a collection of many walker steps are called a chain.

One increasingly popular sampling algorithm is called [Nested Sampling](https://en.wikipedia.org/wiki/Nested_sampling_algorithm) which does not necessarily use an MCMC method. Nested sampling can be difficult to explain intuitively, but one of the nice advantages to nested sampling is that it can be used for model comparison. More relevant to our purposes here, the [dynesty](https://dynesty.readthedocs.io/en/stable/) dynamic nested sampling algorithm is able to quickly sample our posterior and has a convenient stopping condition which ensures the sampler has converged so that we get reliable uncertainties.

⚠️ Note that Stage 5 has TWO control files, one `.ecf`and one `.epf`⚠️

### 1.1 Setting the Stage 5 "Eureka! Control File" (ECF)

**This determines what will happen during Stage 5**

To begin, please first copy below the contents of the ECF template for MIRI/LRS from the `S5_template.ecf` file in the ECF demos folder on [GitHub](https://github.com/kevin218/Eureka/tree/exoslam2025/demos/JWST).

The most important parameters and their recommended settings are described below, but more context can be found on the [Eureka! documentation website](https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-5).

1. Set `ncpu` to the number of CPU threads you want to use. If set to `1` no multiprocessing will be done, and this parameter can be increased to ~2x your CPU core count for faster runs.
3. Set `verbose` to `True` so you get lots of useful information printed out.
4. Set `fit_method` to `[dynesty]` to instead use the dynesty sampler.
5. Set `run_myfuncs` to `[batman_ecl, polynomial, expramp, ypos, ywidth]`. Each element in this list is a function that will be fitted to the data — we'll setup more aspects of each function below when we setup the "Eureka! Parameter File". The `batman_ecl` function uses the [batman](http://lkreidberg.github.io/batman/docs/html/index.html) package to model an eclipse function which describes how the astrophysical flux decreases when we go from seeing the combined light from the star and planet to just the light from the star when the planet is behind the star. The `polynomial` function allows us to fit for systematic trends in time (like a linear slope) as well as fitting for the overall baseline flux level; you must always use a `polynomial` model when fitting observations to at least model the baseline flux level. The `expramp` function allows us to model the detector systematic ramp in MIRI data caused by some combination of detector persistence/settling. The `ypos` and `ywidth` functions model changes in the flux caused by changes in the position and PSF width of the spectrum on the detector in the spatial direction; we computed these during Stage 3 in Notebook #1. JWST is quite stable over time, so these functions just assume that the flux is linearly correlated with the spatial position and PSF width. Not all observations require that you use the `ypos` and `ywidth` functions, but they're often useful for MIRI observations.
6. Set `fit_par` to `./S5_fit_par_template.epf`. This tells Eureka! where you have specified the priors and initial guesses for the parameters the control the functions listed above.
7. Set `manual_clip` to `[[None,975]]` to remove **a lot** of the initial integrations which are worst affected by the initial exponential ramp. You **do not** always remove this many integrations, but we will for this demonstration for multiple reasons.
8. Leave all parameters under the "Limb darkening controls" heading at their default values of `None` since we're fitting an eclipse observation and stellar limb darkening has no impact on eclipse observations.
9. Add the following dynesty-related settings
  1. Set `run_nlive` to `121`. This sets the number of "live" points that dynesty will use (for more details see [here](https://dynesty.readthedocs.io/en/stable/faq.html#live-point-questions)). For reliable results, this must be greater than `ndim * (ndim + 1) // 2`, where `ndim` is the number of fitted parameters and `//` indicates integer division. In general, the larger this number the more robust your results will be and the longer they will take to run. Setting `run_nlive` to a value of `'min'` will make sure the minimum number of live points is respected and is often sufficient for initial analyses, while more in-depth analyses might benefit from using a larger number of live points (like our value of `121`).
  2. Set `run_bound` to `'multi'`. This sets the type of bounds used by dynesty to "multiple ellipsoid decomposition". For more information, see [here](https://dynesty.readthedocs.io/en/stable/faq.html#bounding-questions). A value of `'multi'` should work for most problems.
  3. Set `run_sample` to `'rwalk'`. This is often the optimal sampling method but that sometimes depends on the number of fitted parameters, and `auto` can sometimes also be a safe bet for most problems. For more information, see [here](https://dynesty.readthedocs.io/en/stable/faq.html#sampling-questions).
  4. Finally, set `run_tol` to `0.01`. This determines the stopping condition of the sampler. Smaller values will result in more precise posteriors at the cost of increased runtime, while larger values can result in poorly converged garbage. In general, a value of `0.1` should work for initial investigations, while `0.01` might work better when producing your "final" results.
10. Set `isplots_S5` to `5` so that you see lots of useful plots that can help you debug your fits.
11. Set `nbin_plot` to `None` since we're not fitting that many integrations and we don't really need to temporally bin the data when plotting to be able to see our astrophysical signal.
12. Set `hide_plots` to `False` so that all your figures appear within the notebook (you can set this to `True` if you're running the code from the terminal and want to avoid an excessive number of pop-ups appearing).
13. Set `topdir` to `{path_to_data_folder_on_your_machine}` which will use the folder you specified above.
14. Since we're fitting white lightcurves, we'll adjust the `inputdir` to `Stage4_white` and the `outputdir` to `Stage5_white`. To choose a specific run, use the syntax `Stage4_white/S4_YYYY-MM-DD_miri_lrs_runN` where YYYY is the year (e.g., `2025`), MM is the month (e.g., `04`), DD is the day (e.g., `18`), and N is the particular run number that you want to use (e.g., `1`).

In [ ]:
s5_ecf_contents = f"""

# Fill this f-string text block with the contents of the S5 ECF template
# from https://github.com/kevin218/Eureka/tree/exoslam2025/demos/JWST
# and then adjust the values as described above.

"""

with open(f'S5_{eventlabel}.ecf', 'w') as f:
    f.write(s5_ecf_contents)

### 1.2 Setting the Stage 5 "Eureka! Parameter File" (EPF)

**This determines the priors and starting guesses for your fitted models**

To begin, please first copy below the contents of the EPF template from the `S5_fit_par_template.epf` file in the ECF demos folder on [GitHub](https://github.com/kevin218/Eureka/tree/exoslam2025/demos/JWST).

Here is where we dive further into specifying details about the models that we want to fit to our data. We'll discuss the ones relevant to this fit below, but more details can be found on the [Eureka documentation webpage](https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-5-fit-parameters).

Each row has 6 columns:
> `Name Value Free? PriorPar1 PriorPar2 PriorType`

`Name` specifies the name of a fitted variable. These have to be from a very specific list of options so that Eureka! knows what the variable is supposed to mean. For example `rp` is the planet-to-star radius ratio (so `rp` squared would be equal to the transit depth) and `fp` is the eclipse depth.

`Value` specifies the starting guess for the variable. This column is used for all optimization or sampling algorithms except dynesty which does not take a starting guess. In general, starting guesses do not need to be exceptionally good, and as long as they're in the vague ballpark then the fitting or sampling algorithm should work fine. For fixed parameters (see the description of `Free?` below), this column sets the value of the parameter.

`Free?` specifies whether the parameter should be freely fitted for each wavelength (`'free'`), should be set to a fixed value and not changed throughout the fit (`'fixed'`), is an auxilliary variable that also should not be fit (`'independent'`). Other more advanced options also exist, but we won't cover them here. For fixed and independent variables, the remaining three columns are not used.

**The rightmost column**, `PriorType`, determines what type of prior function you will use to constrain your fitted parameters. The three options are Normal (`N`; also known as a Gaussian prior), Uniform (`U`), and Log-Uniform (`LU`). Some sampling algorithms work faster or more efficiently with Normal priors, but sometimes you must use Uniform priors to avoid unphysical parameter settings. Normal priors are typically best for orbital parameters for which there are published best-fit values and uncertainties.

The meanings of the `PriorPar1` and `PriorPar2` columns are dependent on the setting of the `PriorType` column (which is the last column). If `PriorType` is N, then `PriorPar1` is the mean of the Gaussian prior and `PriorPar2` is the standard deviation. If `PriorType` is U, then `PriorPar1` is the lower limit of the parameter and `PriorPar2` is the upper limit of the parameter. If `PriorType` is LU, then `PriorPar1` is the lower limit of the log of the parameter, and `PriorPar2` is the upper limit of the log of the parameter.

<br/>

---

<br/>

__Our astrophysical models are controlled by the following parameters__:

- For `per` (planetary orbital period), `t0` (time of conjunction; also called linear ephemeris or transit midpoint), `inc` (planetary orbital inclination), `a` (planetary orbital semi-major axis), `ecc` (planetary orbital eccentricity), and `w` (planetary orbital argument of periastron) we will make use of previously published research. In general, one can navigate to the [NASA Exoplanet Archive's page](https://exoplanetarchive.ipac.caltech.edu/overview/WASP-43b) about our target WASP-43b and click on "WASP-43 b Planetary Parameters". However, since these MIRI/LRS data were already published and give more accurate orbital parameters, we will use the values published by [Hammond et al. 2024](https://ui.adsabs.harvard.edu/abs/2024AJ....168....4H/abstract) (Table 1, "Eclipse Map Fit") as priors. Where there are two values for the uncertainty, just use the larger of the two. Also, while the published `t0` is in units of BJD_TDB, ⚠️ JWST timestamps are in units of B**M**JD_TDB ⚠️, where the M means "modified" and BMJD_TDB=BJD_TDB-2400000.5; this removes the first two digits that won't change in our lifetime and changes the time so that a new BMJD_TDB day starts at midnight (while new BJD_TDB days inconventiently start at noon). In general we would need to subtract that 2400000.5 from the published `t0` value when entering it into your EPF, but [Hammond et al. 2024](https://ui.adsabs.harvard.edu/abs/2024AJ....168....4H/abstract) published the `t0` in units of BMJD_TDB already. [Hammond et al. 2024](https://arxiv.org/abs/2404.16488) did not re-fit the orbital period, `per`, and instead just fixed the orbital period to `0.813474056` days from [Kokori et al. 2023](https://ui.adsabs.harvard.edu/abs/2023ApJS..265....4K/abstract); we will do the same. And since the planet is consistent with a zero-eccentricity orbit, let's set `ecc` to be `'fixed'` to `0` and arbitrarily set `w` to be `'fixed'` to `90` (the value of `w` doesn't matter if the eccentricity is zero). Additionally, with only a single eclipse we will likely be unable to significantly improve the constraints on any of these orbital parameters with the exception of the mid-eclipse time, so we will set all these orbital parameters except `t0` to be `'fixed'` to their published values.
- In summary, you should have:
> `per 0.813474056 'fixed'`<br/>
> `t0 55934.292283 'free' 55934.292283 0.000011 N`<br/>
> `inc 82.106 'fixed'`<br/>
> `a 4.859 'fixed'`<br/>
> `ecc 0 'fixed'`<br/>
> `w 90 'fixed'`<br/>

- For `Rs` (the stellar radius in units of Solar radii), we can again use the [NASA Exoplanet Archive's page](https://exoplanetarchive.ipac.caltech.edu/overview/WASP-43b) about WASP-43, this time instead clicking on the "WASP-43 Stellar Parameters" section. This parameter is only used to account for the finite speed of light, so simply fixing it to the published value from [Bonomo et al. 2017](https://exoplanetarchive.ipac.caltech.edu/overview/WASP-43) will work:
> `Rs 0.667 'fixed'`

- For `rp` (planet-to-star radius ratio), we'll use the radius fitted from a later part of these data where the planet transited. Please set `rp` to be fixed to a value of 0.15839 based on the results of [Hammond et al. 2024](https://ui.adsabs.harvard.edu/abs/2024AJ....168....4H/abstract) (Table 1, "Eclipse Map Fit"). Without this knowledge, one could also compute the planet-to-star radius ratio from the planet and star radii published by Bonomo et al. 2017, although care would need to be taken to ensure units were properly accounted for. In summary, set:
> `rp 0.15839 'fixed'`

- Set time_offset to the following to ignore the parameter:
> `time_offset 0 'independent'`.

- For `fp` (eclipse depth) we will have to create a "minimally informative prior". With this prior, we seek to constrain the model to the very rough area that we expect for the parameters while minimally influencing the final results with our semi-arbitrary decision. For `fp`, a visual inspection of the lightcurve produced in the last notebook suggests that the eclipse depth is around 6000 ppm, so a reasonable, "minimally informative" prior could be $6000 \pm 5000$ ppm which would be specified as:
  > `fp 6000e-6 'free' 6000e-6 5000e-6 N`

  where `6000e-6` is a more conventient way of specifying 6000 ppm than carefully counting decimal places and specifying `0.006` (although both options are possible).

__Our systematic models are controlled by__:

  - `c0` (the baseline flux level) and `c1` (which fits for a linear slope as a function of time). These two parameters control the `polynomial` function we specified in the ECF settings above. To make things easier, Eureka! normalizes the lightcurves by the median flux, so the value of `c0` should be approximately 1, but more precisely should be set to the value of the normalized flux during the middle of the eclipse observation; by eye this appears to be around 0.996. The value of `c1` is in the units of change in `c0` per day, and is typically very small (the absolute value of `c1` is typically at or below 0.001. Reasonable, minimally informative priors could then be:
  > `c0 0.996 'free' 0.996 0.01 N`<br/>
  > `c1 0 'free' 0 0.01 N`
  - `r0` (the exponential ramp amplitude) and `r1` (the exponential ramp timescale). These two parameters control the `expramp` function we specified in the ECF settings above. The value of `r0` controls the amplitude of the exponential ramp and should be close to 0 given how many initial integrations we have trimmed. The value of `r1` is the exponential decay rate in units of days<sup>-1</sup>, and is typically in the range of 5-500. Reasonable, minimally informative priors could then be:
  > `r0 0 'free' 0 0.01 N`<br/>
  > `r1 200 'free' 5 500 U`
  - `ypos` which assumes the observed flux varies linearly with the position on the detector. This can happen because of flux lost outside of our extraction aperture or between gaps between pixels. Values for `ypos` are typically well below 1, so a reasonable, minimally informative prior could then be:
  > `ypos 0 'free' 0 0.1 N`
  - and `ywidth` which assumes the observed flux varies linearly with the PSF width of the spectrum. This can happen because of flux lost outside of our extraction aperture or between gaps between pixels. Values for `ywidth` can sometimes reach as high as ~1, so a reasonable, minimally informative prior could then be:
  > `ywidth 0 'free' 0 10 N`

- We also need to setup `scatter_mult`. Eureka! estimates the expected level of noise from photon-limited statistics, but data is imperfect and we rarely ever reach the photon-limited noise level (because of background noise, read noise, and other systematic noise sources). If you fit data with underestimated error bars, then the resulting confidence intervals can be artificially small, so we want to inflate the error bars on our observations as is justified by the data. The simplest way to do this with Eureka! is the `scatter_mult` parameter which multiplies the error bars by a constant factor. A fitted value of 2 for `scatter_mult` would mean that the observed scatter in the data is twice as high as it would be in the photon-limited noise regime. MIRI data is typically pretty well behaved though, and a reasonable, minimally informative prior could be:
> `scatter_mult 1.1 'free' 0.8 10 U`

**Finally, delete or comment-out (with `#`) all other parameters not mentioned above.**

In [ ]:
s5_epf_contents = f"""

# Fill this f-string text block with the contents of the S5 EPF template
# from https://github.com/kevin218/Eureka/tree/exoslam2025/demos/JWST
# and then adjust the values as described above.

"""

with open('S5_fit_par_template.epf', 'w') as f:
    f.write(s5_epf_contents)

### 1.3 Running Eureka!'s Stage 5

The following cell will run Eureka!'s Stage 5 using the settings you defined above. Note that your ECF and EPF will be copied to your output folder, making it easy to remember how you produced those outputs hours, days, or years after you reduced the data.

The following fit should take ~2 minutes to complete for this current setup if `ncpu` was set to `1` (though the exact runtime is stochastic and also depends on the speed of your CPU).

In [ ]:
s5_meta = eureka.S5_lightcurve_fitting.s5_fit.fitlc(eventlabel)

---
## 2. Optimizing fits to spectroscopic lightcurves

Now that we know how to find best-fit values and uncertainties from a single "white" lightcurve, it is easy to extrapolate those skills to perform fits to "spectroscopic" lightcurves to evaluate how parameters like the eclipse depth vary as a function of wavelength (which can tell us about molecules in the planets' atmospheres).

### 2.1 Adjusting the Stage 5 "Eureka! Control File" (__ECF__)

Thankfully we can reuse the vast majority of our ECF settings above with only a few minor tweaks.

To begin, please first copy below the contents of the ECF that you filled out above. Then:

1. Change `fit_method` to `[lsq]` to use the [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) algorithm to **minimize** the **negative** log-probability in order to find the best-fit values for each fitted parameter. Running dynesty on each channel takes quite a long time, so we'll instead just do a quick optimization run with `lsq`.
2. Set `lsq_method` to `Powell`. This controls which scipy.optimize.minimize optimization method will be used. In general, the Powell algorithm works very well.
3. Set `lsq_tol` to `1e-6`. This sets the error tolerance for the scipy.optimize.minimize optimization method, where a smaller value will result in a more precise estimate of the "best-fit" value, so long as the optimizer manages to converge. A value of `1e-6` will suffice for most datasets.
4. Leave `lsq_maxiter` at its default setting of `None`. This parameter can be used to let the optimzation routine run for longer if you get a message saying that the fit failed because the maximum number of function evaluations has been exceeded. This can sometimes happen when `lsq_tol` is set too small, when you're fitting a complex model with many parameters, or when there are strong degeneracies between two or more parameters.
5. Since we're now fitting spectroscopic lightcurves, we'll adjust the `inputdir` to `Stage4` and the `outputdir` to `Stage5`. If you want to choose a specific Stage4 run, you can do so using the syntax `Stage4/S4_YYYY-MM-DD_miri_runN` where YYYY is the year (e.g., `2025`), MM is the month (e.g., `06`), DD is the day (e.g., `09`), and N is the particular run number that you want to use (e.g., `1`).

And that's it for adjustments to the ECF!

In [ ]:
s5_ecf_contents = f"""

# Fill this f-string text block with the contents of the S5 ECF you edited above
# and then adjust the values as described above.

"""

with open(f'S5_{eventlabel}.ecf', 'w') as f:
    f.write(s5_ecf_contents)

### 2.2 Adjusting the Stage 5 "Eureka! Parameter File" (__EPF__)

Thankfully we can reuse the vast majority of our EPF settings above with only a few minor tweaks.

To begin, please first copy below the contents of the EPF that you filled out above. Then you just need to set any parameters that cannot vary with wavelength to be `'fixed'`. The relevant parameters that cannot vary with wavelength are the orbital parameters which are `per`, `t0`, `a`, and `inc`; of these, our previous fit only used `t0`.

One option is to fix these parameters to the best-fit values obtained from your optimization run on the white light curve. Alternatively, you could also fix them to the published values from [Bonomo et al. 2017](https://exoplanetarchive.ipac.caltech.edu/overview/WASP-43). We'll go with the former, so copy-paste your best-fit value for `t0` from your above fit into the `Value` column and set the `Free?` column to `'fixed'`.

In [ ]:
s5_epf_contents = f"""

# Fill this f-string text block with the contents of the S5 EPF you edited above
# and then adjust the values as described above.

"""

with open('S5_fit_par_template.epf', 'w') as f:
    f.write(s5_epf_contents)

### 2.3 Running Eureka!'s Stage 5

The following cell will run Eureka!'s Stage 5 using the settings you defined above. Note that your ECF and EPF will be copied to your output folder, making it easy to remember how you produced those outputs hours, days, or years after you reduced the data.

The code will loop through each wavelength channel, printing information about the fit and showing plots as it goes.

The following fits should take a total of ~1 minute to complete for the 11 channels. While waiting for the fits to complete, you could begin setting up the next section.

In [ ]:
s5_meta = eureka.S5_lightcurve_fitting.s5_fit.fitlc(eventlabel)

---
## 3. Stage 6 - emission and transmission spectra

Now that we've completed fits to the 11 spectroscopic channels, we want an easy way of seeing how different parameters vary with wavelength. Of particular interest is the transit depth `rp`, the eclipse depth `fp` (also referred to as day-side flux, when the phase is 0.5), as well as the night-side flux `fn` (when the phase is 0). The variations in day-side and night-side as a function of wavelength is called an emission spectrum. The variations in transit depth as a function of wavelength is called a transmission spectrum. Both transmission and emission spectra can be used to determine the presence and abundance of molecules in the atmosphere of an exoplanet. Because we only used the `lsq` optimization algorithm above, our emission spectrum will not have error bars. If you want to know what the spectrum looks like with error bars, you'll have to re-run your spectroscopic fits above using a sampler like `dynesty`.

### 3.1 Setting the Stage 6 "Eureka! Control File" (ECF)

**This determines what will happen during Eureka!'s Stage 6**

To begin, please first copy below the contents of the ECF template for MIRI/LRS from the `S6_template.ecf` file in the ECF demos folder on [GitHub](https://github.com/kevin218/Eureka/tree/exoslam2025/demos/JWST).

The most important parameters and their recommended settings are described below, but more context can be found on the [Eureka! documentation website](https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-6).

1. Set `y_params` to `['fp']`. Eureka! is able to plot any of the `'free'` parameters from your fits as a function of wavelength, but the only two we're really interested in here are `fp`.
2. Set `y_scalars` to `[1e6]` so that units of ppm are used for a plot that is easier to read.
3. You can safely ignore the settings in the "Scale height parameters" chunk and "Model" chunk as they are not relevant for our purposes.
4. Set `topdir` to `{path_to_data_folder_on_your_machine}` which will use the folder you specified above.
5. Since we're dealing with spectroscopic lightcurve fits, we'll adjust the `inputdir` to `Stage5` and the `outputdir` to `Stage6`. If you want to choose a specific Stage5 run, you can do so using the syntax `Stage5/S5_YYYY-MM-DD_miri_runN` where YYYY is the year (e.g., `2025`), MM is the month (e.g., `06`), DD is the day (e.g., `09`), and N is the particular run number that you want to use (e.g., `1`).

That's all we need to change for our purposes here, and all the other parameters can be left at their default settings.

In [ ]:
s6_ecf_contents = f"""

# Fill this f-string text block with the contents of the S6 ECF template
# from https://github.com/kevin218/Eureka/tree/exoslam2025/demos/JWST
# and then adjust the values as described above.

"""

with open(f'S6_{eventlabel}.ecf', 'w') as f:
    f.write(s6_ecf_contents)

### 3.2 Running Eureka!'s Stage 6

The following cell will run Eureka!'s Stage 6 using the settings you defined above. Note that your ECF will be copied to your output folder, making it easy to remember how you produced those outputs hours, days, or years after you reduced the data.

The following stage should take &#8810;1 minute to complete.

In [ ]:
s6_meta = eureka.S6_planet_spectra.s6_spectra.plot_spectra(eventlabel)

Observation: As you can see from your emission spectrum, the eclipse depth increases with wavelength. This is generally true of thermal emission spectra because the emission spectrum is approximately a ratio of blackbody functions which increases with increasing wavelength. You should also see a decrease in the eclipse depth around 6.5 microns - this feature is indicative of water in the atmosphere of WASP-43b. To get error bars in the y-direction, you'll need to fit your spectroscopic lightcurves using a sampler like dynesty rather than the lsq fitter we used for expedited fitting here.

---